## Pipeline & Batch Processing for Coloured Images

In [ ]:
def enhance_pipeline(img):
    # Step 1: Global exposure adjustment
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(4,4))
    l_clahe = clahe.apply(l)

    lab_clahe = cv2.merge((l_clahe, a, b))
    img = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

    # Step 2: White balance correction
    result = img.copy().astype(np.float32)
    avg_b, avg_g, avg_r = np.mean(result[:, :, 0]), np.mean(result[:, :, 1]), np.mean(result[:, :, 2])
    avg_gray = (avg_b + avg_g + avg_r) / 3
    result[:, :, 0] *= avg_gray / avg_b
    result[:, :, 1] *= avg_gray / avg_g
    result[:, :, 2] *= avg_gray / avg_r
    img = np.clip(result, 0, 255).astype(np.uint8)

    # Step 3: Local tone mapping
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    l_filtered = cv2.bilateralFilter(l, d=7, sigmaColor=75, sigmaSpace=75)
    lab_filtered = cv2.merge((l_filtered, a, b))
    img = cv2.cvtColor(lab_filtered, cv2.COLOR_LAB2BGR)

    # Step 4: Color enhancement
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV).astype(np.float32)
    h, s, v = cv2.split(hsv)
    mean_saturation = np.mean(s)
    if mean_saturation < 100:
        sat_boost = 1.5 if mean_saturation < 70 else 1.2
    else:
        sat_boost = 1.0
    s = np.clip(s * sat_boost, 0, 255)
    hsv_enhanced = cv2.merge([h, s, v])
    img = cv2.cvtColor(hsv_enhanced.astype(np.uint8), cv2.COLOR_HSV2BGR)

    blurred_original = cv2.GaussianBlur(img, (9, 9), 0)
    detail_layer = cv2.addWeighted(img, 1.5, blurred_original, -0.5, 0)
    img = cv2.addWeighted(img, 0.8, detail_layer, 0.5, 0)

    # Step 5: Sharpening
    blurred = cv2.GaussianBlur(img, (0, 0), 1.2)
    alpha = 1
    img = cv2.addWeighted(img, 1 + alpha, blurred, -alpha, 0)

    return img

# ---------- BATCH PROCESS ----------
def batch_process_images(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    image_paths = glob(os.path.join(input_folder, "*.*"))
    print(f"Found {len(image_paths)} images to process.")

    for img_path in image_paths:
        img = cv2.imread(img_path)
        if img is None:
            print(f"Skipping file (not an image): {img_path}")
            continue

        output_img = enhance_pipeline(img)

        base_name = os.path.basename(img_path)
        name, ext = os.path.splitext(base_name)
        output_path = os.path.join(output_folder, f"{name}_output{ext}")
        cv2.imwrite(output_path, output_img)
        print(f"Processed & saved: {output_path}")

input_folder = r"PATH/TO/FOLDER"
output_folder = r"PATH/TO/FOLDER"
batch_process_images(input_folder, output_folder)

## Input & Output Image Comparison on Browser

In [ ]:
import os
import webbrowser

input_folder = r"PATH/TO/FOLDER"
output_folder = r"PATH/TO/FOLDER"
html_file = "comparison_01.html"

# Allowed image extensions
IMAGE_EXTS = {".png", ".jpg", ".jpeg"}

def is_image(fname):
    return os.path.splitext(fname)[1].lower() in IMAGE_EXTS

input_images = [f for f in os.listdir(input_folder) if is_image(f)]
output_images = [f for f in os.listdir(output_folder) if is_image(f)]

output_map = {}
for out_img in output_images:
    name, ext = os.path.splitext(out_img)
    if name.lower().endswith("_output"):
        base = name[:-7]
    else:
        base = name
    output_map[base.lower()] = out_img  

# HTML content
html_content = """
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Image Comparison</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background: #f4f4f4;
            padding: 20px;
        }
        .pair {
            display: flex;
            justify-content: center;
            align-items: flex-start;
            margin-bottom: 30px;
            background: #fff;
            padding: 15px;
            border-radius: 8px;
            box-shadow: 0px 2px 6px rgba(0,0,0,0.2);
        }
        .image-container {
            text-align: center;
            margin: 0 15px;
        }
        img {
            max-width: 400px;
            height: auto;
            border-radius: 6px;
            border: 1px solid #ccc;
        }
        .caption {
            margin-top: 8px;
            font-weight: bold;
        }
        .missing {
            color: red;
            font-weight: bold;
        }
    </style>
</head>
<body>
    <h1>Input vs Output Image Comparison</h1>
"""

for in_img in sorted(input_images, key=str.lower):
    base, _ = os.path.splitext(in_img)
    out_img = output_map.get(base.lower()) 
    html_content += f"""
    <div class="pair">
        <div class="image-container">
            <img src="{input_folder}/{in_img}" alt="Input Image">
            <div class="caption">Input: {in_img}</div>
        </div>
    """
    if out_img:
        html_content += f"""
        <div class="image-container">
            <img src="{output_folder}/{out_img}" alt="Output Image">
            <div class="caption">Output: {out_img}</div>
        </div>
        """
    else:
        html_content += f"""
        <div class="image-container">
            <div class="missing"> No matching output found</div>
        </div>
        """
    html_content += "</div>"

html_content += """
</body>
</html>
"""

# Save HTML file
with open(html_file, "w", encoding="utf-8") as f:
    f.write(html_content)

webbrowser.open("file://" + os.path.abspath(html_file))

print(f"HTML file generated and opened: {html_file}")

## Pipeline & Batch Processing for Dull Images

In [ ]:
import os
import cv2
import numpy as np

input_folder = r"PATH/TO/FOLDER"
output_folder = r"PATH/TO/FOLDER"

os.makedirs(output_folder, exist_ok=True)

# PIPELINE 
def contrast_stretch(img):
    img_float = img.astype(np.float32)
    min_val = np.min(img_float)
    max_val = np.max(img_float)
    stretched = ((img_float - min_val) / (max_val - min_val)) * 255
    return stretched.astype(np.uint8)

def gamma_correction(img, gamma=0.6):
    lut = np.array([((i / 255.0) ** gamma) * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(img, lut)

def white_balance(img):
    result = img.copy().astype(np.float32)
    avg_b = np.mean(result[:, :, 0])
    avg_g = np.mean(result[:, :, 1])
    avg_r = np.mean(result[:, :, 2])
    avg_gray = (avg_b + avg_g + avg_r) / 3

    result[:, :, 0] *= avg_gray / avg_b
    result[:, :, 1] *= avg_gray / avg_g
    result[:, :, 2] *= avg_gray / avg_r

    result = np.clip(result, 0, 255).astype(np.uint8)
    return result

def clahe_enhancement(img, clipLimit=2.0, tileGridSize=(4, 4)):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    L, A, B = cv2.split(lab)

    clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)
    L_eq = clahe.apply(L)

    lab_eq = cv2.merge((L_eq, A, B))
    return cv2.cvtColor(lab_eq, cv2.COLOR_LAB2BGR)

def color_enhancement(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV).astype(np.float32)
    h, s, v = cv2.split(hsv)

    mean_saturation = np.mean(s)
    if mean_saturation < 40:
        sat_boost = 1.8
    elif mean_saturation < 70:
        sat_boost = 1.5
    elif mean_saturation < 100:
        sat_boost = 1.2
    else:
        sat_boost = 1.0

    s = np.clip(s * sat_boost, 0, 255)
    hsv_enhanced = cv2.merge([h, s, v])
    return cv2.cvtColor(hsv_enhanced.astype(np.uint8), cv2.COLOR_HSV2BGR)

def process_pipeline(img):
    img = contrast_stretch(img)
    img = gamma_correction(img, gamma=0.6)
    img = white_balance(img)
    img = clahe_enhancement(img, clipLimit=2.0, tileGridSize=(4, 4))
    img = color_enhancement(img)
    return img

# BATCH PROCESSING
for file in os.listdir(input_folder):
    if file.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff")):
        input_path = os.path.join(input_folder, file)

        img = cv2.imread(input_path)
        if img is None:
            continue
        result = process_pipeline(img)

        name, ext = os.path.splitext(file)
        output_filename = f"{name}_output{ext}"
        output_path = os.path.join(output_folder, output_filename)
        cv2.imwrite(output_path, result)

        print(f"Processed: {file} -> {output_filename}")

print("Batch processing completed!")

## Input & Output Image Comparison on Browser

In [ ]:
import os
import webbrowser

input_folder = r"PATH/TO/FOLDER"
output_folder = r"PATH/TO/FOLDER"
html_file = "dull_images_comparison.html"

IMAGE_EXTS = {".png", ".jpg", ".jpeg"}

def is_image(fname):
    return os.path.splitext(fname)[1].lower() in IMAGE_EXTS

input_images = [f for f in os.listdir(input_folder) if is_image(f)]
output_images = [f for f in os.listdir(output_folder) if is_image(f)]

output_map = {}
for out_img in output_images:
    name, ext = os.path.splitext(out_img)
    if name.lower().endswith("_output"):
        base = name[:-7]  
    else:
        base = name
    output_map[base.lower()] = out_img  

# HTML content
html_content = """
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Image Comparison</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background: #f4f4f4;
            padding: 20px;
        }
        .pair {
            display: flex;
            justify-content: center;
            align-items: flex-start;
            margin-bottom: 30px;
            background: #fff;
            padding: 15px;
            border-radius: 8px;
            box-shadow: 0px 2px 6px rgba(0,0,0,0.2);
        }
        .image-container {
            text-align: center;
            margin: 0 15px;
        }
        img {
            max-width: 400px;
            height: auto;
            border-radius: 6px;
            border: 1px solid #ccc;
        }
        .caption {
            margin-top: 8px;
            font-weight: bold;
        }
        .missing {
            color: red;
            font-weight: bold;
        }
    </style>
</head>
<body>
    <h1>Input vs Output Image Comparison</h1>
"""

for in_img in sorted(input_images, key=str.lower):
    base, _ = os.path.splitext(in_img)
    out_img = output_map.get(base.lower()) 
    html_content += f"""
    <div class="pair">
        <div class="image-container">
            <img src="{input_folder}/{in_img}" alt="Input Image">
            <div class="caption">Input: {in_img}</div>
        </div>
    """
    if out_img:
        html_content += f"""
        <div class="image-container">
            <img src="{output_folder}/{out_img}" alt="Output Image">
            <div class="caption">Output: {out_img}</div>
        </div>
        """
    else:
        html_content += f"""
        <div class="image-container">
            <div class="missing">No matching output found</div>
        </div>
        """
    html_content += "</div>"

html_content += """
</body>
</html>
"""

# Save HTML file
with open(html_file, "w", encoding="utf-8") as f:
    f.write(html_content)

webbrowser.open("file://" + os.path.abspath(html_file))

print(f"HTML file generated and opened: {html_file}")